# 测试通过嵌入检索的成功率

初步结论：

- k=1时，只有56%，明显低于中文提示词的情况，后者是72%
- k=2时，达到83%，又高于中文的情况
- k=5时，依然是83%

初始情况不如中文提示词，但是文档摘要中包括问答。

整体看documentSummaryIndex，简单易用，但是效果一般，只能达到70%左右的检索召回率

## 准备

In [1]:
%%time

INDEX_PATH="retrieve-index"
DATA_PATH="retrieve-data"

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.34 µs


In [2]:
%%time
!rm -rf $INDEX_PATH

CPU times: user 2.54 ms, sys: 4.35 ms, total: 6.89 ms
Wall time: 109 ms


In [3]:
%%time

# test_data=[
#     {
#         "url": "https://www.guancha.cn/ZiZheng/2024_06_27_739401_s.shtml",
#         "question": "和平共处五项原则为啥到现在还有重要意义？"
#     },
#     {
#         "url": "https://www.guancha.cn/kegongliliang/2024_06_27_739408_s.shtml",
#         "question": "丰鸟科技在内蒙古主场地的资产清点和撤出的具体原因是什么？"
#     },
#     {
#         "url": "https://user.guancha.cn/main/content?id=1257033",
#         "question": "电池工厂火灾的具体起因是什么？"
#     },
#     {
#         "url": "https://user.guancha.cn/main/content?id=1257049",
#         "question": "莫迪计划推出的亲商改革具体包括哪些措施？"
#     },
#     {
#         "url": "https://www.guancha.cn/politics/2024_06_26_739354.shtml",
#         "question": "金融监管总局支持信托公司和理财公司加大创业投资力度的具体措施有哪些？"
#     },
#     {
#         "url": "https://www.guancha.cn/gaotianwei/2024_06_24_739062_s.shtml",
#         "question": "我们为什么要发展可重复使用运载火箭的验证箭？"
#     },
#     {
#         "url": "https://www.guancha.cn/economy/2024_06_26_739352.shtml",
#         "question": "中小银行合并重组的原因是什么？"
#     }
# ]

test_data=[
    {
        "url": "https://www.guancha.cn/ZiZheng/2024_06_27_739401_s.shtml",
        "question": "和平共处五项原则为啥到现在还有重要意义？"
    },
    {
        "url": "https://www.guancha.cn/kegongliliang/2024_06_27_739408_s.shtml",
        "question": "丰鸟科技在内蒙古主场地的资产清点和撤出的具体原因是什么？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257033",
        "question": "电池工厂火灾的具体起因是什么？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257049",
        "question": "莫迪计划推出的亲商改革具体包括哪些措施？"
    },
    {
        "url": "https://www.guancha.cn/politics/2024_06_26_739354.shtml",
        "question": "金融监管总局支持信托公司和理财公司加大创业投资力度的具体措施有哪些？"
    },
    {
        "url": "https://www.guancha.cn/gaotianwei/2024_06_24_739062_s.shtml",
        "question": "我们为什么要发展可重复使用运载火箭的验证箭？"
    },
    {
        "url": "https://www.guancha.cn/economy/2024_06_26_739352.shtml",
        "question": "中小银行合并重组的原因是什么？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739492.shtml",
        "question": "除了菲律宾，日本政府还计划在哪些东南亚国家推广5G建设计划？"
    },
    {
        "url": "https://www.guancha.cn/politics/2024_06_27_739480.shtml",
        "question": "尤勇智对年轻演员有何建议？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257405",
        "question": "俄罗斯如何对待塞尔维亚总统武契奇的表态？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739443_s.shtml",
        "question": "梅洛尼在欧盟高层人选提名问题上的态度是什么？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739432.shtml",
        "question": "金砖国家为什么决定暂时停止接纳新成员国？"
    },
    {
        "url": "https://www.guancha.cn/internation/2024_06_27_739441.shtml",
        "question": "印尼旅游业面临的竞争压力有哪些？"
    },
    {
        "url": "https://www.guancha.cn/XueKaiHuan/2024_06_24_739023_s.shtml",
        "question": "波兰近年来在对华政策上采取的“双轨政策”具体体现在哪些方面？"
    },
    {
        "url": "https://www.guancha.cn/economy/2024_06_27_739420.shtml",
        "question": "为了加强技能人才培养并解决结构性就业矛盾，国务院常务会议部署了哪些具体措施？"
    },
    {
        "url": "https://user.guancha.cn/main/content?id=1257296",
        "question": "以色列内塔尼亚胡所述的“激战阶段”如何定义？"
    },
    {
        "url": "https://www.guancha.cn/ShiYang/2024_06_23_738937_s.shtml",
        "question": "菲律宾为何选择派遣运补船和充气艇非法进入中国南沙群岛仁爱礁邻近海域？"
    },
    {
        "url": "https://www.guancha.cn/fengchao/2024_06_23_738933_s.shtml",
        "question": "普京访越期间，俄罗斯和越南就哪些领域的合作达成了新的协议或共识？"
    }
]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 5.96 µs


In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [4]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    data = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    data['url']=url
    return data

CPU times: user 136 ms, sys: 111 ms, total: 247 ms
Wall time: 102 ms


In [5]:
%%time

import json

for news in test_data:
    data=get_news_data(news['url'])
    data['url']=news['url']
    file_path = f'./{DATA_PATH}/{data["title"]}.json'
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

CPU times: user 2 s, sys: 16.1 ms, total: 2.01 s
Wall time: 4.62 s


In [6]:
%%time

from llama_index.core import(
    Document
)

def data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2.55 s, sys: 200 ms, total: 2.75 s
Wall time: 2.56 s


In [7]:
%%time

from llama_index.core import SimpleDirectoryReader

documents=SimpleDirectoryReader(input_dir=f"./{DATA_PATH}").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

CPU times: user 18.3 ms, sys: 0 ns, total: 18.3 ms
Wall time: 3.07 s


In [8]:
%%time

import json

docs=[]
for document in documents:
    news_data=json.loads(document.text) # documents[0].text
    docs.append(data2doc(news_data))

documents=docs

len(docs)

CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 1.64 ms


18

In [9]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.42 ms, sys: 119 µs, total: 1.54 ms
Wall time: 1.37 ms


In [10]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings

Settings.llm=get_llm()
Settings.embed_model=get_embedding()

CPU times: user 655 ms, sys: 35.9 ms, total: 691 ms
Wall time: 691 ms


In [11]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/18 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/18 [00:00<?, ?it/s]

current doc id: “塞铁”跳反了？塞尔维亚“背刺”俄罗斯：承认已暗中向乌克兰提供8亿美元弹药！
current doc id: “对抗中国技术”，日本计划帮菲律宾建设5G通信网络
current doc id: 一周军评：菲律宾“后继乏力”，我们的考验又在哪里？
current doc id: 丁刚：中印能实现龙象共舞吗？
current doc id: 中东又一场战争可能要爆发了！
current doc id: 中小银行改革加速度，今年来超80家开启合并重组模式
current doc id: 冯超：普京访越，透视出的国家相处之道应该是什么？
current doc id: 印尼考虑对华免签，预计10月份前敲定
current doc id: 国常会：一视同仁支持内外资企业参与大规模设备更新、政府采购和投资
current doc id: 子政：和平共处五项原则历久弥新的原因是什么？
current doc id: 尤勇智：不可想象！现在素人、网红、有背景的人都能来演戏
current doc id: 拉夫罗夫：金砖十国已投票决定，暂时停止扩员以专注整合新成员国
current doc id: 欧盟“内定”冯德莱恩连任，意大利总理警告：注定在欧洲议会遇到困难
current doc id: 科工力量：前有高铁，后有快递小哥，支线物流在中国飞不起来？
current doc id: 薛凯恒：日渐亲美的波兰，是否会改变一贯的对华中立政策？
current doc id: 金融监管总局：将研究提高保险资金投资创业投资基金集中度的比例上限
current doc id: 韩国电池厂火灾17名中国人遇难：有多少同胞，在韩国打工？
current doc id: 高天伟：国产可重复火箭完成10公里测试，有一点与马斯克很不一样


Generating embeddings:   0%|          | 0/18 [00:00<?, ?it/s]

CPU times: user 1.08 s, sys: 58.6 ms, total: 1.14 s
Wall time: 6min 52s


In [12]:
%%time

doc_summary_index.storage_context.persist(INDEX_PATH)

CPU times: user 52.9 ms, sys: 8.1 ms, total: 61 ms
Wall time: 60.2 ms


In [26]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=INDEX_PATH)
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 102 ms


## 检索

### 简单测试

In [27]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=1,
)

retrieved_nodes = retriever.retrieve(test_data[0]["question"])

CPU times: user 6.63 ms, sys: 45 µs, total: 6.67 ms
Wall time: 107 ms


In [28]:
retrieved_nodes[0].metadata

{'title': '冯超：普京访越，透视出的国家相处之道应该是什么？',
 'publish_time': '2024-06-23 08:48:43',
 'author': '小婷',
 'url': 'https://www.guancha.cn/fengchao/2024_06_23_738933_s.shtml',
 'images': ['https://i.guancha.cn/news/dfic/2024/06/21/20240621150313664.jpg',
  'https://i.guancha.cn/news/dfic/2024/06/21/20240621150436659.jpg',
  'https://i.guancha.cn/shiping-banner.jpg']}

In [29]:
retrieved_nodes[0].metadata['url']==test_data[0]["url"]

False

### k=1

In [30]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=1,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        # print(news["question"])
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results

CPU times: user 87.6 ms, sys: 450 µs, total: 88 ms
Wall time: 1.7 s


[False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True]

In [31]:
results.count(True) / len(results)

0.5555555555555556

### k=2

In [32]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=2,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 101 ms, sys: 476 µs, total: 101 ms
Wall time: 1.7 s


0.8333333333333334

### k=3

In [19]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=3,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 113 ms, sys: 608 µs, total: 114 ms
Wall time: 1.73 s


0.8333333333333334

### k=5

In [20]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=5,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 132 ms, sys: 4.88 ms, total: 137 ms
Wall time: 1.75 s


0.8333333333333334

### k=6

In [21]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=6,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 133 ms, sys: 12.4 ms, total: 146 ms
Wall time: 1.71 s


0.8888888888888888

### k=len(test_data)

In [22]:
len(test_data)

18

In [23]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=len(test_data),
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 291 ms, sys: 0 ns, total: 291 ms
Wall time: 1.91 s


1.0

In [24]:
doc_summary_index.get_document_summary("丁刚：中印能实现龙象共舞吗？")

'The provided texts discuss the potential for cooperation between India (referred to as "the elephant") and China (referred to as "the dragon") in various aspects, particularly focusing on their economic relations and manufacturing sectors. The discussions highlight the ambitions of Indian Prime Minister Narendra Modi\'s government to boost India\'s manufacturing capabilities and compete with China, which is seen as a major competitor.\n\nThe texts delve into several challenges that India faces in achieving its goals:\n\n1. **Education and Skill Levels**: There are concerns about the quality of education and skills among the workforce, particularly regarding issues like caste systems that limit opportunities for certain groups.\n2. **Gender Participation**: The low participation rate of women in the labor market is mentioned as a significant challenge.\n3. **Infrastructure Issues**: Problems such as land acquisition and minority rights are discussed as barriers to economic development.

In [25]:
%%time

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    similarity_top_k=len(test_data)/2,
)

results=[]

for news in test_data:
    retrieved_nodes = retriever.retrieve(news["question"])
    result=False
    for node in retrieved_nodes:
        if node.metadata['url']==news["url"]:
            result=True
            break
    results.append(result)

results.count(True) / len(results)

CPU times: user 180 ms, sys: 619 µs, total: 181 ms
Wall time: 1.8 s


0.8888888888888888